# Projeto 1

## Componentes

- Raphael Ramos
- Ernane Ferreira

## Dataset

Usamos um dos datasets das Requisições de Materiais no grupo de Materiais, dos Dados Abertos da UFRN. Abaixo uma descrição dele

In [437]:
# Importa a biblioteca pandas com o alias 'pd'
import pandas as pd

# Lê um arquivo CSV como delimitador ;
rm = pd.read_csv('requisicao-material-2019.csv', delimiter=';')

# Exibe as primeiras linhas do DataFrame 'rm' para visualizar os dados
rm.head()

,numero,ano,requisicao,data,id_unidade_requisitante,unidade_requisitante,id_unidade_custo,unidade_custo,observacoes,grupo_material,convenio,status,tipo_requisicao,almoxarifado,valor
0,35420,2019,35420/2019,31/12/2019,8031,PROAD - COORDENADORIA DE GESTÃO DA INFORMAÇÃO,30545,MCTIC - ATUALIZAÇÃO DA INFRAESTRUTURA TECNOLÓGICA,ATUALIZAÇÃO DA INFRAESTRUTURA TECNOLÓGICA PARA...,EQUIPAMENTOS DE PROCESSAMENTOS DE DADOS,False,LIQUIDADA,SOLICITAÇÃO DE MATERIAL DE UM REGISTRO DE PREÇOS,NaN,"R$ 984.295,00"
1,35419,2019,35419/2019,31/12/2019,8031,PROAD - COORDENADORIA DE GESTÃO DA INFORMAÇÃO,30545,MCTIC - ATUALIZAÇÃO DA INFRAESTRUTURA TECNOLÓGICA,ATUALIZAÇÃO DA INFRAESTRUTURA TECNOLÓGICA PARA...,EQUIPAMENTOS DE PROCESSAMENTOS DE DADOS,False,EM_LIQUIDACAO,SOLICITAÇÃO DE MATERIAL DE UM REGISTRO DE PREÇOS,NaN,"R$ 16.344,00"
2,35418,2019,35418/2019,30/12/2019,1423,DIRETORIA DE LOGÍSTICA - PROAD,9184,PROGRAMA DE CONSOLIDAÇÃO DAS IFES,NaN,EQUIPAMENTOS DE PROCESSAMENTOS DE DADOS,False,LIQUIDADA,SOLICITAÇÃO DE MATERIAL DE UM REGISTRO DE PREÇOS,NaN,"R$ 15.290,00"
3,35417,2019,35417/2019,30/12/2019,1423,DIRETORIA DE LOGÍSTICA - PROAD,9184,PROGRAMA DE CONSOLIDAÇÃO DAS IFES,NaN,EQUIPAMENTOS DE PROCESSAMENTOS DE DADOS,False,LIQUIDADA,SOLICITAÇÃO DE MATERIAL DE UM REGISTRO DE PREÇOS,NaN,"R$ 27.490,00"
4,35416,2019,35416/2019,30/12/2019,1451,SUSTENTAÇÃO DE ESTOQUE DO ALMOXARIFADO CENTRAL,1451,SUSTENTAÇÃO DE ESTOQUE DO ALMOXARIFADO CENTRAL,NaN,MATERIAL DE LIMPEZA E PRODUTOS DE HIGIENIZACAO,False,LIQUIDADA_PARCIALMENTE,SOLICITAÇÃO DE MATERIAL DE UM REGISTRO DE PREÇOS,NaN,"R$ 3.065,00"


## Contexto

Quando uma unidade precisa de algum material, ela abre uma requisição para o almoxarifado. Nessa requisição, deve ter o preço estimado desse material e a unidade que vai custear esse material. Assim, o almoxarifado pode aprová-la e dar início a ordem de compra.

## Ideia

Neste dataset, queremos que, dada uma requisição, prever ele vai ser negada ou autorizada pelo almoxarifado. Além disso, investigar qual perfil das requisições que têm mais chances de serem aprovadas.

## Preprocessamento

Como esse dataset possui registros de devolução e requisições que envolvem serviços, vamos considerar apenas as requisições de material da instituição

In [438]:
# Filtre apenas as requisicoes de materiais
df = rm[rm['tipo_requisicao'] == 'REQUISIÇÃO DE MATERIAL']

Algumas colunas em nada agregam ao nosso objetivo, portanto vamos retirá-las

In [439]:
# Retirar colunas numero, ano, requisicao,data e observacoes
df = df.drop(['numero','ano','requisicao','data','observacoes','grupo_material','tipo_requisicao','unidade_custo','id_unidade_custo'], axis=1)

Como a coluna unidade_requisitante possui identificador inteiro, vamos eliminá-la

In [440]:
# Retirar colunas unidade_requisitante e unidade_custo
df = df.drop(['unidade_requisitante'], axis=1)

A coluna status é multivalorada, mas apenas nos importa saber se a requisição foi negada ou não.

In [441]:
df = df[df['status'].isin(['NEGADA', 'AUTORIZADA', 'LIQUIDADA', 'EM_LIQUIDACAO', 'COMPRA'])]

As colunas convenio e status são interessantes, mas são categóricas. Portanto, precisão ser tratadas. Abaixo uma função que facilita esse trabalho

In [442]:
from sklearn.preprocessing import LabelEncoder

def categorize(dataframe, column):
    """
    Recebe um dataframe e a coluna com os dados categóricos que se quer rotular

    Args:
        dataframe (object DataFrame)
        column (string)
    """
    # Identificar categorias de column
    data = {category for category in set(dataframe[column])} 
    
    # Traduzir cada categoria para numeros
    label_encoder = LabelEncoder()
    
    # Atualiza no dataframe
    dataframe[column] = label_encoder.fit_transform(dataframe[column])
    
    # Obtenha as categorias originais e seus rótulos numéricos
    categories = label_encoder.classes_
    labels = label_encoder.transform(categories)

    # Visualizar mapeamento
    display(pd.DataFrame({'Categoria':categories, 'Rótulo Numérico': labels}))

In [443]:
categorize(df, 'status')

,Categoria,Rótulo Numérico
0,COMPRA,0
1,EM_LIQUIDACAO,1
2,LIQUIDADA,2
3,NEGADA,3


In [444]:
categorize(df, 'convenio')

,Categoria,Rótulo Numérico
0,False,0


In [445]:
categorize(df, 'almoxarifado')

,Categoria,Rótulo Numérico
0,ALMOXARIFADO - CCHLA,0
1,ALMOXARIFADO - CT,1
2,ALMOXARIFADO CENTRAL,2
3,ALMOXARIFADO DE MATERIAIS DE MANUTENÇÃO DE IMÓ...,3
4,NaN,4


Abaixo está o dataset após o preprocessamento

In [446]:
df

,id_unidade_requisitante,convenio,status,almoxarifado,valor
781,6026,0,3,3,"R$ 355,84"
782,6026,0,3,3,"R$ 120,92"
1030,205,0,1,4,"R$ 2.490,00"
1032,19,0,3,2,"R$ 81,10"
1178,1433,0,3,2,"R$ 88,30"
...,...,...,...,...,...
31226,8772,0,3,2,"R$ 1.053,44"
32636,1433,0,3,3,"R$ 3.563,44"
33039,8864,0,3,2,"R$ 600,00"
33188,8772,0,3,2,"R$ 20,00"
